This Jupyter notebook contains a set of sample codes that are necessary for obtaining the calculation results of three different datasets as described in the paper titled "Functional Output Regression for Machine Learning in Materials Science". Readers can use the code to perform model training and test with full hyperparameter search, which will take a long time. Instead, readers can also test the performance of the best performing models that are downloadable from our github server. 

We provide three pre-trained models for each datasets.
- Dataset 1: https://github.com/yoshida-lab/XenonPy/releases/download/v0.6.3/pretrained_models.zip
- Dataset 2: https://github.com/yoshida-lab/XenonPy/releases/download/v0.6.3/dataset2.zip
- USGS: https://github.com/yoshida-lab/XenonPy/releases/download/v0.6.3/USGS.zip

Please update the variable "model_dir" in the "User parameter setting" section accordingly.

To obtain the datasets used in this study, please access the following references.
- For Dataset I and Dataset II:
Urbina et al. (2021) UV-adVISor: Attention-Based Recurrent Neural Networks to Predict UV–Vis Spectra. Analytical Chemistry, 93(48), 16076-16085.
(https://pubs.acs.org/doi/10.1021/acs.analchem.1c03741)
- USGS:
Kokaly et al. (2017) USGS Spectral Library Version 7 Data: U.S. Geological Survey data release (https://dx.doi.org/10.5066/F7RR1WDJ)

For Dataset I and II, please directly pick the files "Datasets I, II, III spectra and SMILES/Dataset_I.csv" and "Datasets I, II, III spectra and SMILES/Dataset_II.csv" in the downloaded zip file, respectively, as the input csv files. For USGS dataset, please process the downloaded data using "process_dataset_usgs.ipynb" (https://github.com/yoshida-lab/XenonPy/blob/master/samples/process_dataset_usgs.ipynb) to obtain the compact csv files used in our study. A file named "Dataset_USGS.csv" shall be produced after running the codes in the notebook. If there is any trouble processing the data, please contact us for help.


In [1]:
# python packages used in the sample codes

from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.utils.data
from torch import optim

from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import Draw
from rdkit import rdBase, Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.AtomPairs import Pairs, Torsions

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# user-friendly print out
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


RDKit WARNING: [23:24:15] Enabling RDKit 2019.09.1 jupyter extensions
[23:24:15] Enabling RDKit 2019.09.1 jupyter extensions


# Preparation

#### Utility function: define RBFKernel class to be the radial basis function kernel that will be used in our models

In [2]:
import numpy as np

from typing import Union, Sequence, Callable, Tuple


class RBFKernel():
    def __init__(self, sigmas_squared: Union[float, int, np.ndarray, Sequence], hight=10, *, dtype='float32'):
        """
        Radial Basis Function (RBF) kernel function.
        Ref: https://en.wikipedia.org/wiki/Radial_basis_function_kernel

        Parameters
        ----------
        sigmas:
            The squared standard deviations (SD).
            Can be a single number or a 1d array-like object.
        x_i: np.ndarray
            Should be a 1d array.
        x_j: np.ndarray
            Should be a 1d array.

        Returns
        -------
        np.ndarray
            Distribution under RBF kernel.

        Raises
        ------
        ValueError
            Raise error if sigmas has wrong dimension.
        """
        sigmas_squared = np.asarray(sigmas_squared)
        if sigmas_squared.ndim == 0:
            sigmas_squared = sigmas_squared[np.newaxis]
        if sigmas_squared.ndim != 1:
            raise ValueError('parameter `sigmas_squared` must be a array-like object which has dimension 1')
        self._sigmas_squared = sigmas_squared
        self.hight = hight
        self.dtype = dtype
        
    @property
    def sigmas_squared(self):
        return np.copy(self._sigmas_squared)
    
    def __call__(self, x_i: np.ndarray, x_j: Union[np.ndarray, int, float]):
        # K(x_i, x_j) = exp(-||x_i - x_j||^2 / (2 * sigma^2))
        p1 = np.power(np.expand_dims(x_i, axis=x_i.ndim) - x_j, 2)
        p2 = self._sigmas_squared * 2
        dists = self.hight * np.exp(-np.expand_dims(p1, axis=p1.ndim) / p2)

        if dists.shape[2] == 1:
            return dists[:, :, 0].astype(self.dtype)
        return dists.astype(self.dtype)


#### Model definition: define KernelNet class to be the neural network based kernel function model

In [3]:
from xenonpy.model import SequentialLinear

# model parameter initialization
@torch.no_grad()
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)

        
class KernelNet(nn.Module):
    def __init__(self, *,
                 n_neurons=(1024, 896, 512, 256, 128),
                 activation_func=nn.LeakyReLU(0.2, inplace=True),
                 kernel_grids: Union[int, np.ndarray] = 128,
                 wavelength_points: Union[int, np.ndarray] =181,
                 kernel_func=RBFKernel(sigmas_squared=0.05, hight=1),
                ):
        super().__init__()

        # initializing fixed params (not updated through training)
        if isinstance(kernel_grids, int):
            kernel_grids = np.linspace(0, 1, kernel_grids)
        if isinstance(kernel_grids, np.ndarray):
            self.kernel_grids = kernel_grids
        else:
            raise ValueError('kernel_grids error!')
        
        if isinstance(wavelength_points, int):
            wavelength_points = np.linspace(0, 1, wavelength_points)
        if isinstance(wavelength_points, np.ndarray):
            self.wavelength_points = wavelength_points
        else:
            raise ValueError('wavelength_points error!')
        
        # fully connected layers
        self.fc_layers = SequentialLinear(
            in_features=n_neurons[0], out_features=n_neurons[-1], h_neurons=n_neurons[1:-1], h_activation_funcs=activation_func
        )
        
        # baseline parameters
        self.mu = torch.nn.Parameter(torch.zeros(wavelength_points.size))
        
        # kernel
        self.kernel = torch.from_numpy(
            kernel_func(self.kernel_grids, self.wavelength_points)
        )
        
    def to(self, *arg, **kwarg):
        self.kernel = self.kernel.to(*arg, **kwarg)
        return super().to(*arg, **kwarg)

    def forward(self, fingerprint_input):
        x = self.fc_layers(fingerprint_input)
        x = x.view(x.size(0), self.kernel_grids.size, -1) 
        x = x * self.kernel
        x = torch.sum(x, dim=1)
        x = self.mu + x
        return x
    

# User parameter setting

In [4]:
# Perform best case only or not: True - best model only; False - all hyperparameter search
best_case = True

# Target dataset: 1 - Dataset I; 2 - Dataset II; 3 - USGS
data_case = 1

# Full directory of the processed data file
data_file = "Dataset_I.csv"

# Directory and folder name for saving all the outputs
out_directory = "spectrum_results"

# Directory and folder name for saving or loading of models
if data_case == 1:
    model_dir = 'model/dataset1'
elif data_case == 2:
    model_dir = 'model/dataset2'
elif data_case == 3:
    model_dir = 'model/usgs'
    

# Main codes

### Loading data

In [5]:
# load csv file
input_file = pd.read_csv(data_file,sep=",")

# Figure
out_result = Path(out_directory)
out_result.mkdir(exist_ok=True, parents=True)

# drop "OC(=O)C(=C\C1=CC=[Cl]C=[Cl]1)\C1=CN=CC=C1" because of Explicit valence error (only relevant for Dataset I)
input_file = input_file[input_file.SMILES != "OC(=O)C(=C\C1=CC=[Cl]C=[Cl]1)\C1=CN=CC=C1"]
# drop data containing NAN　because of Explicit valence error (only relevant for Dataset II)
input_file = input_file.dropna()

file_smiles_spectrum = input_file.values
smiles_list = (file_smiles_spectrum[:,1])
spectrum_list = file_smiles_spectrum[:,2:]
spectrum_list = np.array(spectrum_list, dtype='float32')

print("number of data:", len(smiles_list))


number of data: 949


#### convert smiles to fingerprint

In [6]:
# convert smiles to mol file
mol_list = [Chem.MolFromSmiles(s) for s in smiles_list if s is not None]
    
# Morgan fingerprint (ECFP) with radius=3 and 1024 bits
fps_bit = [AllChem.GetMorganFingerprintAsBitVect(m, 3, 1024) for m in mol_list]
fps_list = []
for fps in fps_bit:
    fps_arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fps, fps_arr)
    fps_list.append(fps_arr)
fps_list = np.array(fps_list, dtype='float32')

fps_list_max = np.amax(fps_list)
fps_list_min = np.amin(fps_list)
fps_norm = ((fps_list - fps_list_min) / (fps_list_max - fps_list_min))
X_data = torch.from_numpy(fps_norm)

# process spectrum data
spectrum_list_max = np.amax(spectrum_list)
spectrum_list_min = np.amin(spectrum_list)
spectrum_norm = ((spectrum_list - spectrum_list_min) / (spectrum_list_max - spectrum_list_min))
Y_data = torch.from_numpy(spectrum_norm)
data_number = torch.zeros((X_data.shape[0],1))
Y_data = torch.cat((data_number, Y_data), dim=1)
for i in range(Y_data.shape[0]):
    number = torch.tensor([i]) 
    Y_data[i][0] = number
    
# show data size
X_data.shape
Y_data.shape


torch.Size([949, 1024])

torch.Size([949, 182])

### Set up hyper-params

#### 1. model training parameters

In [7]:
n_epoch = 3000 # epoch
lr = 0.0002 # learning rate
ngpu = 2 # number of GPU
display_interval = 100

model_path = Path(model_dir)
model_path.mkdir(exist_ok=True, parents=True)


In [8]:
# use GPU if available
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# split train:val:test 
if data_case == 1:
    split_size = 0.316 # DatasetI:0.316, DatasetII:0.301, USGS:0.2
    split_val_size = 0.5 # DatasetI:0.5, DatasetII:0.488, USGS:0.5
elif data_case == 2:
    split_size = 0.301 # DatasetI:0.316, DatasetII:0.301, USGS:0.2
    split_val_size = 0.488 # DatasetI:0.5, DatasetII:0.488, USGS:0.5
elif data_case == 3:
    split_size = 0.2 # DatasetI:0.316, DatasetII:0.301, USGS:0.2
    split_val_size = 0.5 # DatasetI:0.5, DatasetII:0.488, USGS:0.5


In [9]:
# indexing corresponding to model parameters below
if best_case:
    if data_case == 1:
        start_layer = 3
        end_layer = 4
        start_hyper = 7
        end_hyper = 8
        wavelength_points = 181
        kernel_grids = 128
    elif data_case == 2:
        start_layer = 2
        end_layer = 3
        start_hyper = 7
        end_hyper = 8
        wavelength_points = 171
        kernel_grids = 128
    elif data_case == 3:
        start_layer = 2
        end_layer = 3
        start_hyper = 6
        end_hyper = 7
        wavelength_points = 2151
        kernel_grids = 256
else:
    start_layer = 0
    end_layer = 4
    start_hyper = 0
    end_hyper = 12
    

#### 2. model parameters

In [10]:
# the number of hidden layers
if data_case == 3:
    n_neurons = [
        (1024, 640, 256),                 # DatasetUSGS: 1 hidden layer
        (1024, 768, 512, 256),            # DatasetUSGS: 2 hidden layers
        (1024, 832, 640, 448, 256),       # DatasetUSGS: 3 hidden layers
        (1024, 873, 718, 564, 410, 256),  # DatasetUSGS: 4 hidden layers
    ]
else:
    n_neurons = [
        (1024, 512, 128),                 # DatasetI,II: 1 hidden layer
        (1024, 512, 256, 128),            # DatasetI,II: 2 hidden layers
        (1024, 896, 512, 256, 128),       # DatasetI,II: 3 hidden layers
        (1024, 896, 640, 512, 256, 128),  # DatasetI,II: 4 hidden layers
    ]
    
# the length and variance of RBF kernel
kernel_hypers = (
    [10, 0.00005], [5, 0.00005], [1, 0.00005], [0.5, 0.00005],
    [10, 0.0005], [5, 0.0005], [1, 0.0005], [0.5, 0.0005],
    [10, 0.00025], [5, 0.00025], [1, 0.00025], [0.5, 0.00025],
)


### Begin model training

In [11]:
# fixing random seeds for reproducing the results
np.random.seed(0)
torch.manual_seed(0)


#### train with different hyperparameters

In [ ]:
if (data_case == 1 or data_case == 2):
    batch_number = 50
elif (data_case == 3):
    batch_number = 9

# repeat for different number of layers in the neural network
for layer_number in range(start_layer, end_layer):
    # repeat for different sigma values for the kernel
    for hyper_number in range(start_hyper, end_hyper):
        # repeat for three times to check sensitivity to data splitting
        for random_number in range(3):
            # split data with different random seed
            X_train, X_test_val, Y_train, Y_test_val = train_test_split(X_data, Y_data, test_size=split_size, random_state=random_number) 
            X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=split_val_size, random_state=random_number)
            Dataset_train = torch.utils.data.TensorDataset(X_train, Y_train)
            
            loader_train = torch.utils.data.DataLoader(Dataset_train, batch_size=batch_number, shuffle=True)
            Dataset_val = torch.utils.data.TensorDataset(X_val, Y_val)
            loader_val = torch.utils.data.DataLoader(Dataset_val, shuffle=False)

            # bulid model
            netG = KernelNet(
                n_neurons=n_neurons[layer_number],
                wavelength_points=wavelength_points,
                kernel_grids=kernel_grids,
                kernel_func=RBFKernel(
                    sigmas_squared=kernel_hypers[hyper_number][1],
                    hight=kernel_hypers[hyper_number][0]
                ),
            ).to(device)

            # Handle multi-gpu if desired

            # if (device.type == 'cuda') and (torch.cuda.device_count() > ngpu):
            #     netG = nn.DataParallel(netG, device_ids=range(ngpu))            
            netG = netG.apply(weights_init)

            # setup optimizer for training model
            regu = torch.Tensor([1]).to(device) # parameter regularization
            criterion = nn.MSELoss()
            optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999), weight_decay=1e-5)
            
            # train with fixed number of epochs
            losses = []
            print(f"training {layer_number + 1} hidden layer model with hyper_number {hyper_number + 1} and random_number {random_number}...")
            
            for epoch in range(n_epoch):
                for itr, data in enumerate(loader_train):
                    real_fps = data[0][:,:].to(device)
                    real_spectrum = data[1][:,1:].to(device)
                    optimizerG.zero_grad()

                    pred_spectrum = netG(real_fps)

                    mu_parameter = netG.mu
                    mu_para_diff = torch.diff(mu_parameter)
                    delta_mu = torch.sum(torch.pow((mu_para_diff),2))
                    
                    loss = criterion(pred_spectrum, real_spectrum[:,:]) + regu * delta_mu

                    loss.backward()
                    optimizerG.step()
                    losses.append(loss.item())

                    if epoch % display_interval == 0 and itr % display_interval == 0:
                        print('[{}/{}][{}/{}] Loss: {:.7f} '.format(epoch + 1, n_epoch, itr + 1, len(loader_train), loss.item()))
                        
            # save model
            PATH_G = '{}/generator_time{:03d}_layer{:03d}_hyper{:03d}.pth'.format(model_path, random_number, layer_number+1, hyper_number+1) 
            torch.save(netG.state_dict(), PATH_G)


### Test model performance

In [12]:
# repeat for different number of layers in the neural network
for layer_number in range(start_layer, end_layer):
    # repeat for different sigma values for the kernel
    for hyper_number in range(start_hyper, end_hyper):
        # initializing variables for performance evaluation
        anal_index = []        
        anal_real_list = []
        anal_pred_list = []

        peak_position_real_list = []
        peak_position_pred_list = []
        peak_intensity_real_list = []
        peak_intensity_pred_list = []

        rmse_median = []
        rmse_data = []
        rsquare_median = []
        mae_median = []
        drmse_median = []
        
        # repeat for three times to check sensitivity to data splitting
        for random_number in range(3):
            # split data with different random seed
            X_train, X_test_val, Y_train, Y_test_val = train_test_split(X_data, Y_data, test_size=split_size, random_state=random_number) 
            X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=split_val_size, random_state=random_number)
            Dataset_train = torch.utils.data.TensorDataset(X_train, Y_train)
            loader_train = torch.utils.data.DataLoader(Dataset_train, batch_size=50, shuffle=True)
            Dataset_val = torch.utils.data.TensorDataset(X_val, Y_val)
            loader_val = torch.utils.data.DataLoader(Dataset_val, shuffle=False)

            # bulid model
            netG = KernelNet(
                n_neurons=n_neurons[layer_number],
                wavelength_points=wavelength_points,
                kernel_grids=kernel_grids,
                kernel_func=RBFKernel(
                    sigmas_squared=kernel_hypers[hyper_number][1],
                    hight=kernel_hypers[hyper_number][0]
                ),
            ).to(device)
    
            #########
            ## Load trained model
            #########
            trained_netG_path = '{}/generator_time{:03d}_layer{:03d}_hyper{:03d}.pth'.format(model_path, random_number, layer_number+1, hyper_number+1)
            netG.load_state_dict(torch.load(trained_netG_path))     

            ############
            ### Analysis
            ############
            pred_spectrum = netG(X_test.to(device))
            anal_real = Y_test[:,1:].to('cpu').detach().numpy().copy()
            anal_pred = pred_spectrum[:].to('cpu').detach().numpy().copy()
            anal_index.extend(Y_test[:,0].to('cpu').detach().numpy().copy())
            anal_real_list.extend(anal_real)
            anal_pred_list.extend(anal_pred)

            rmse_list = []
            rsquare_list = []
            mae_list = []
            drmse_list = []
        
            for i in range(anal_real.shape[0]):
                # RMSE
                rmse = np.sqrt(mean_squared_error(anal_real[i], anal_pred[i]))
                rmse_list.append(rmse)
                rmse_data.append(rmse)
                # R square
                rsquare = r2_score(anal_real[i], anal_pred[i])
                rsquare_list.append(rsquare)
                # MAE
                mae = mean_absolute_error(anal_real[i], anal_pred[i])
                mae_list.append(mae)
            # RMSE derivative
            real_d = anal_real[:,:-1] - anal_real[:,1:]
            pred_d = anal_pred[:,:-1] - anal_pred[:,1:]
            drmse = np.sqrt(np.sum(((real_d - pred_d) ** 2), axis=1) / anal_real.shape[0])
            drmse_list.append(drmse)
            # calculate median
            rmse_median.append(np.median(rmse_list))
            rsquare_median.append(np.median(rsquare_list))
            mae_median.append(np.median(mae_list))
            drmse_median.append(np.median(drmse_list))

        # all test results
        anal_index = np.array(anal_index, dtype='int32')
        anal_real_list = np.array(anal_real_list, dtype='float32')
        anal_pred_list = np.array(anal_pred_list, dtype='float32')


        print("#####################")
        print("##     output results  ##")
        print("#####################")
        print(netG)

        # output profiles
        device_cpu = torch.device('cpu')
        exp = pd.DataFrame(anal_real_list)
        exp.insert(loc = 0, column='smiles', value= smiles_list[anal_index])
        if data_case == 1:
            exp.to_csv('{}/exp_profiles_dataset1.csv'.format(out_result))
        elif data_case == 2:
            exp.to_csv('{}/exp_profiles_dataset2.csv'.format(out_result))
        elif data_case == 3:
            exp.to_csv('{}/exp_profiles_USGS.csv'.format(out_result))
        pred = pd.DataFrame(anal_pred_list)
        pred.insert(loc = 0, column='smiles', value= smiles_list[anal_index])
        if data_case == 1:
            pred.to_csv('{}/pred_profiles_dataset1.csv'.format(out_result))
        elif data_case == 2:
            pred.to_csv('{}/pred_profiles_dataset2.csv'.format(out_result))
        elif data_case == 3:
            pred.to_csv('{}/pred_profiles_USGS.csv'.format(out_result))
            
        # RMSE        
        rmse_df = pd.DataFrame([rmse_data])
        rmse_df = rmse_df.transpose()
        rmse_df.columns = ["rmse"]
        rmse_sort = rmse_df.sort_values('rmse')
        rmse_sort_index = rmse_sort.index.values
        print("Median of RMSE:", np.mean(rmse_median), "std:", np.std(rmse_median))
        # R2
        print("Median of R square:", np.mean(rsquare_median), "std:", np.std(rsquare_median))
        # MAE
        print("Median of MAE:", np.mean(mae_median), "std:",np.std(mae_median))            
        # dRMSE
        print("Median of RMSE_derivative", np.mean(drmse_median), "std:",np.std(drmse_median))
        

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

#####################
##     output results  ##
#####################
KernelNet(
  (fc_layers): SequentialLinear(
    (layer_0): LinearLayer(
      (linear): Linear(in_features=1024, out_features=896, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (normalizer): BatchNorm1d(896, eps=0.1, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_1): LinearLayer(
      (linear): Linear(in_features=896, out_features=640, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (normalizer): BatchNorm1d(640, eps=0.1, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (layer_2): LinearLayer(
      (linear): Linear(in_features=640, out_features=512, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (normalizer): BatchNorm1d(512, eps=0.1, momentum=0.1, affine=True, track_running_stats=True)
      (activatio